In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25331360
paper_name = 'jiang_zhang_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/fyr12220-sup-0003-TableS2.xlsx', sheet_name='Sheet1', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 109 x 4


In [10]:
original_data.head()

,Systemic,Standard,Unnamed: 2,Unnamed: 3
0,YAL002W,VPS8$*,NaN,NaN
1,YAL012W,CYS3*&,NaN,NaN
2,YAL021C,CCR4#*&,NaN,NaN
3,YBR126C,TPS1,NaN,NaN
4,YBR182C,SMP1,NaN,NaN


In [11]:
original_data['orf'] = original_data['Systemic'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                      Systemic  \
index_input                                                      
106                                                        NaN   
107                                                        NaN   
108          * indicates 54 genes identified from the previ...   

                                                      Standard  \
index_input                                                      
106                                                        NaN   
107                                                        NaN   
108          $ indicates 33 genes identified from the previ...   

                                                    Unnamed: 2  \
index_input                                                      
106                                                        NaN   
107                                                        NaN   
108          & indicates 49 genes identified from the previ...   

       

In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data['data'] = -1

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(106, 1)

# Load & process tested strains

In [21]:
tested = pd.read_excel('raw_data/DELETION LIBRARY.xlsx', sheet_name='DELETION LIBRARY', skiprows=1)

In [22]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,RG NOTES,QC NOTES
0,30338,YAL068C,HOM DIP,01_1,301.0,A,2.0,NaN,NaN
1,30339,YAL067C,HOM DIP,01_1,301.0,A,3.0,NaN,NaN
2,30340,YAL066W,HOM DIP,01_1,301.0,A,4.0,NaN,NaN
3,30341,YAL065C,HOM DIP,01_1,301.0,A,5.0,NaN,NaN
4,30345,YAL062W,HOM DIP,01_1,301.0,A,6.0,NaN,NaN


In [23]:
tested['orf'] = tested['ORF name'].astype(str)

In [24]:
tested['orf'] = clean_orf(tested['orf'])

In [27]:
tested.loc[tested['orf']=='YELOO1C','orf'] = 'YEL001C'

In [28]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [29]:
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name   strain batch  plate row  column RG NOTES  \
index_input                                                                   
346               30827   YMR41W  HOM DIP  13_5  304.0   F     6.0      NaN   

            QC NOTES     orf  
index_input                   
346              NaN  YMR41W  


In [30]:
tested = tested.loc[t,:]

In [31]:
tested_orfs = np.unique(tested['orf'].values)

In [32]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YPR484W']

In [33]:
# Add 1 missing strain to tested
tested_orfs = list(tested_orfs) + missing

In [34]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [771]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,771
data_type,value
orf,
YAL002W,-1
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,771
,data_type,value
gene_id,orf,
2,YAL002W,-1
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        771         
data_type       value   valuez
gene_id orf                   
2       YAL002W    -1 -6.68545
1863    YAL004W     0  0.00000
4       YAL005C     0  0.00000
5       YAL007C     0  0.00000
6       YAL008W     0  0.00000

# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25331360...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.33s/it]

Updating the data_modified_on field...
